# 📘 함수형 프로그래밍 2강: 이터러블·이터레이터와 지연 평가(lazy)

---
## 🎯 학습 목표

- 이터러블(iterable)과 이터레이터(iterator)의 차이를 구분한다.
- `iter()`/`next()` 프로토콜을 이해하고 직접 사용한다.
- 제너레이터와 제너레이터 표현식으로 **지연 평가**를 구현한다.
- 메모리/성능 관점에서 지연 평가의 이점을 설명한다.
---
## 🧩 핵심 개념

이터러블은 반복 가능한 ‘컨테이너’ 인터페이스이고, 이터레이터는 실제로 값을 하나씩 **지연해 산출**하는 객체다.
제너레이터(함수/표현식)는 이터레이터를 손쉽게 만드는 파이썬 도구로, 필요할 때만 값을 계산하여 메모리를 절약한다.
### 핵심 키워드

iterable, iterator, __iter__, __next__, generator, yield, lazy, generator expression

---
## 📝 예시: 리스트 vs 제너레이터의 메모리 차이
같은 범위의 수열을 준비하되, `list(range(...))`와 `range(...)`(제너레이터처럼 지연)로 비교해 본다.


In [1]:
from sys import getsizeof

N = 10_000

eager_list = list(range(N))      # 즉시 전체 생성(eager)
lazy_range = range(N)            # 필요할 때 산출(lazy 유사)

print("list(range(N)) size :", getsizeof(eager_list))
print("range(N)        size :", getsizeof(lazy_range))

# 일부 값만 확인(지연 구조는 전체를 만들지 않아도 된다)
print("first five from range:", [x for x in lazy_range[:5]])


list(range(N)) size : 80056
range(N)        size : 48
first five from range: [0, 1, 2, 3, 4]


---
## 🔧 `iter`/`next` 프로토콜과 제너레이터 소개
- 모든 이터러블은 `iter(obj)`로 이터레이터를 얻고, `next(it)`로 값을 하나씩 소비한다.
- 제너레이터 함수는 `yield`로 값을 순차 산출하며, 자연스럽게 이터레이터를 만든다.
- 제너레이터 표현식 `(f(x) for x in xs)` 역시 지연 계산을 제공한다.


In [5]:
def countdown(n):
    """제너레이터: n, n-1, ..., 1 을 지연 산출"""
    while n > 0:
        yield n
        n -= 1

# iter/next 수동 소비
data = [10, 20, 30]
it = iter(data)
print(next(it))  # 10
print(next(it))  # 20
print(next(it))  # 30
try:
    print(next(it))  # StopIteration
except StopIteration:
    print("StopIteration 발생")

# 제너레이터 사용
for x in countdown(3):
    print("count:", x)

# 제너레이터 표현식
squares_of_even = (x*x for x in range(1, 11) if x % 2 == 0)
print(list(squares_of_even))


10
20
30
StopIteration 발생
count: 3
count: 2
count: 1
[4, 16, 36, 64, 100]


---
## 🧪 실습: 지연 vs 즉시 — 파라미터 바꿔가며 체험하기
아래 코드를 실행해 본 뒤, 상단의 파라미터를 바꾸어 **지연 평가**의 동작을 체감해 보세요.
- 바꿔볼 것: `N`, `LIMIT`, `PRED`, `TRANSFORM`
- 확인할 것: (1) 미리보기 결과, (2) 호출 횟수(필터/변환), (3) 전체 materialize 시의 메모리 차이


In [7]:
import itertools as it
from sys import getsizeof

# ===== 실습 파라미터 =====
N = 1_000_000          # 소스 수열 크기
LIMIT = 8              # 앞에서 몇 개만 소비할지
def PRED(x):           # 필터 기준
    return x % 3 == 0
def TRANSFORM(x):      # 변환 함수
    return x * x
# ========================

# 계측용 래퍼(호출 횟수 추적)
class Calls:
    pred = 0
    trans = 0

def pred_counting(x):
    Calls.pred += 1
    return PRED(x)

def transform_counting(x):
    Calls.trans += 1
    return TRANSFORM(x)

# 지연 파이프라인
src = (n for n in range(N))
filtered = filter(pred_counting, src)
mapped = map(transform_counting, filtered)

preview = list(it.islice(mapped, LIMIT))
print("[Lazy] preview:", preview)
print("[Lazy] pred calls:", Calls.pred, "transform calls:", Calls.trans)

# 같은 작업을 '즉시' 전부 만든 뒤 앞에서 LIMIT개만 보는 경우(비교용)
# 주의: N이 아주 크면 메모리 사용이 급증할 수 있음. 안전을 위해 N_eager를 축소함.
N_eager = min(N, 50_000)
eager_all = [TRANSFORM(x) for x in range(N_eager) if PRED(x)]
print("[Eager] built-size:", len(eager_all), "bytes:", getsizeof(eager_all))
print("[Eager] first LIMIT:", eager_all[:LIMIT])


[Lazy] preview: [0, 9, 36, 81, 144, 225, 324, 441]
[Lazy] pred calls: 22 transform calls: 8
[Eager] built-size: 16667 bytes: 136632
[Eager] first LIMIT: [0, 9, 36, 81, 144, 225, 324, 441]


---
## ⚡ 정리
* 이터러블과 이터레이터는 프로토콜(\_\_iter\_\_, \_\_next\_\_)로 구분된다.
* 제너레이터/표현식은 지연 계산으로 메모리 사용을 줄인다.
* `iter()`/`next()`로 수동 제어가 가능하며, `for` 루프는 이를 자동으로 사용한다.
* `itertools`는 지연 파이프라인을 구성하는 강력한 도구를 제공한다.
---
## 📝 Assignment
1. **`Countdown(n)` 이터레이터 클래스 구현**: `__iter__`와 `__next__`를 직접 정의하여 `n, n-1, ..., 1`을 지연 산출하세요. `iter/next`와 `for` 루프 모두로 동작을 확인하세요.
2. **`batched(iterable, k)` 제너레이터 구현**: 입력 이터러블을 길이 `k`의 **튜플**로 묶어 지연 산출하세요(마지막 배치는 길이가 `k`보다 작을 수 있음). 무한 이터러블(`itertools.count`)과도 안전하게 동작해야 합니다.
3. **`take(n, it)`, `drop(n, it)` 제너레이터 구현**: 각각 앞의 `n`개만 산출, 앞의 `n`개를 건너뛴 나머지를 산출하도록 작성하세요. 리스트 변환 없이 `itertools.islice` 또는 `iter/next`를 활용하세요.
> ✨ 규칙: 리스트로의 전체 변환을 금지하고, **iter/next 프로토콜·제너레이터(yield)·itertools**를 사용해 **지연 평가**를 유지하세요. 무한 이터러블과 함께 테스트해도 멈추지 않도록 주의하세요.
